In [ ]:
# Install necessary libraries
%pip install transformers openai torch tqdm numpy datasets accelerate matplotlib wandb

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached wandb-0.17.5-py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.8 MB)
  Using cached sentry_sdk-2.11.0-py2.py3-none-any.whl (303 kB)
  Using cached setproctitle-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
# Imports
import logging
logging.basicConfig(level='ERROR')
import numpy as np
from pathlib import Path
# import openai
import torch
import zlib
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, AutoModel
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
from eval import *
from experiment_utils import *
from model_utils import *
from unlearning import *
from accelerate import init_empty_weights, infer_auto_device_map

[2024-07-27 11:39:47,546] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
model = 'EleutherAI/pythia-2.8b'
output_dir = f"experiment_output/{model}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

experiment_args = ExperimentArgs(
        model = model,
        output_dir = output_dir,
        # Directory in which to store models locally (to prevent having to download for each experiment)
        model_dir_prefix = "/dbfs/mnt/ds-data-apps/maris/base_models_dupe/",
        data = 'Cyproxius/GutenbergMIA_temporal',
        data_name = 'GutenbergMIA',
        length = 128,
        threshold = 160
    )
# swj0419/WikiMIA
# Cyproxius/GutenbergMIA_temporal
# GutenbergMIA
# NIH_ExPorterMIA_temporal
# NIH_ExPorterMIA
unlearning_args = UnlearningArgs(
        lr=1e-6,
        steps = 4,
        batch_size = 10,
        include_learning = False,
        metric = 'All', # Choose from PPL, Min_K, Min_K++ or All
        num_repeats = 1
    )

Path(output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
PG_analysis_experiment = Experiment(experiment_args, unlearning_args)
PG_analysis_experiment.run_gutenberg_analysis()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:27: UserWarning: This dataset can not be stored in DBFS because either `cache_dir` or the environment variable `HF_DATASETS_CACHE` is set to a non-DBFS path. If this cluster restarts, all saved dataset information will be lost.
  warnings.warn(
/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Creating dataloaders for all permutations
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Creating new dataloader
Finished creating 16 dataloaders
Running experiment for combination ('Adventure stories', 'Adventure stories')


0it [00:00, ?it/s]/Workspace/Users/maris.koopmans@rtl.nl/MIA-metric-thesisproject/src/eval.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(sentence).unsqueeze(0).to(gpu)
1it [00:13, 13.45s/it]

In [ ]:
# # lrs = [1e-7, 5e-6, 1e-6, 5e-5, 1e-5]
# # TODO evaluate all datasets for these parameters with 5x repetitions
# lrs = [1e-6]
# steps = [4]
# batch_sizes = [16]
# # batch_sizes = [1,8,16]
# experiment2 = Experiment(experiment_args, unlearning_args)
# experiment2.run_gridsearch(lrs,steps,batch_sizes)

In [ ]:
import json
import matplotlib.pyplot as plt
graph_dir = output_dir + '/graphs/'
if not os.path.exists(graph_dir):
  os.makedirs(graph_dir)

i = 1
# Load JSON data from the file
with open(f'{output_dir}/{experiment_args.data_name}_length{experiment_args.length}_grid_search_{i}.json', 'r') as file:
    data = json.load(file)

metrics = [metric for metric in data[0].keys() if metric != 'params']

# Extract steps and corresponding MIM_AUC values
steps = [entry['params']['steps'] for entry in data]

all_metric_means = {metric: [entry[metric]['mean'] for entry in data] for metric in metrics}
all_metric_stds = {metric: [entry[metric]['std'] for entry in data] for metric in metrics}

# Plot the data
plt.figure(figsize=(10, 6))
for metric, result in all_metric_means.items():
    plt.errorbar(steps, result, yerr=all_metric_stds[metric], label=metric, marker='o')

# Add titles and labels
plt.title('Gradient ascent-based membership inference')
plt.xlabel('Steps')
plt.ylabel('MIM AUC')
plt.legend()

# Show the plot
plt.grid(True)
# plt.savefig(graph_dir+f'metrics_vs_ULsteps_grid_search_{i}.png')
plt.savefig(graph_dir+f'{experiment_args.data_name}_length{experiment_args.length}_{unlearning_args.metric}.png')

In [ ]:
# sentence = "The quick brown fox jumped over the lazy dog"
# sentence_encode = tokenizer.encode(sentence)
# PPL = calculatePerplexity(sentence_encode, base_model, tokenizer, unlearning_args.device)
# print(f"PPL: {PPL}")